## Process Based Concurrency and creating and starting child processes

A couple of items to note before working with concurrency, parellism, threats and processes (each application is a process and has a single default main thread - task).
- `Concurrency` means doing things out of order i.e. executing tasks out of order
- `Parellelism` executing two tasks at the same time
- `GIL` threading is limited by the Global Interpretor lock
- `IPC` inter-process connectivity

### Multiprocessing vs threading

Similarities:

1. Both Modules are used for concurrency 
2. Similar APIs
3. Support for concurrency primitives in Python

Differences: 
1. Native threads vs Native processes
2. Shared memory vs IPCs (Inter process communications)
3. Limited vs Full Parallelism (GIL)

Generally, use the `multiprocessing` package for CPU bound tasks and not for input / output (IO) bound tasks. 

### Creating and starting a process

- `Main process` = default process created to execute a Python program, has the name *MainProcess*.
- `Main thread` = default thread created by a main process in a Python program, has the name *MainThread*


In [1]:
from time import sleep
from multiprocessing import Process
help(Process)

Help on class Process in module multiprocessing.context:

class Process(multiprocessing.process.BaseProcess)
 |  Process(group=None, target=None, name=None, args=(), kwargs={}, *, daemon=None)
 |  
 |  Method resolution order:
 |      Process
 |      multiprocessing.process.BaseProcess
 |      builtins.object
 |  
 |  Methods inherited from multiprocessing.process.BaseProcess:
 |  
 |  __init__(self, group=None, target=None, name=None, args=(), kwargs={}, *, daemon=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  close(self)
 |      Close the Process object.
 |      
 |      This method releases resources held by the Process object.  It is
 |      an error to call this method if the child process is still running.
 |  
 |  is_alive(self)
 |      Return whether process is alive
 |  
 |  join(self, timeout=None)
 |      Wait until child process terminates
 |  
 |  kill(self)
 |      Terminate process;

In [5]:
%%writefile create_process.py
from time import sleep
from multiprocessing import Process

def task(): 
    sleep(1)
    print('This is from another process', flush=True)

if __name__=='__main__':
    process = Process(target=task)
    process.start()
    print('Waiting for the process to finish')
    process.join()

Writing create_process.py


## Extend the process class

In [1]:
%%writefile process_class_extend.py
from time import sleep
from multiprocessing import Process

class CustomProcess(Process):
    def run(self):
        sleep(1)
        print('This is another process', flush=True)
    def __str__(self):
        return 'This is another process'

# Create entrypoint script in Python
if __name__ == '__main__':
    process = CustomProcess()
    print(process)
    process.start()
    print('Waiting for the process to finish')
    process.join()

Writing process_class_extend.py


## Configure and interact with processes

The below shows how to name a process:

In [2]:
%%writefile name_process.py
from multiprocessing import Process

if __name__=='__main__':
    process = Process(name='FunkyProcess')
    print(process.name)

Writing name_process.py


Configure whether the process is a Daemon or not:

In [3]:
%%writefile check_daemon_proc.py
from multiprocessing import Process

if __name__== '__main__':
    proc = Process(daemon=True)
    print(proc.daemon)

Writing check_daemon_proc.py


### Query the status of a process